In [1]:
from pprint import pprint
from typing import Dict, List, Tuple

import brightway2 as bw
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
import pandas as pd
import productivity_boosters as prbs
from ipysankeywidget import SankeyWidget
from ipywidgets import Layout

In [2]:
bw.projects
bw.projects.set_current("GAC_project")
bw.databases

Databases dictionary with 3 object(s):
	GAC_project_db
	biosphere3
	ei39cutoff

In [3]:
ipcc_methods = [
    met
    for met in bw.methods
    if "ipcc 2021" in str(met).lower()
    and "climate change: including SLCFs" in str(met)
    and "no LT" not in str(met)
    and "GWP100" in str(met)
    or ("ipcc 2013" in str(met).lower() and "no LT" not in str(met) and "GWP100" in str(met))
    # or ("ipcc 2021" in str(met).lower() and "no LT" not in str(met) and "GWP100" in str(met))
][1]# [:2]
ipcc_methods


('IPCC 2021',
 'climate change: including SLCFs',
 'global warming potential (GWP100)')

In [4]:
DB_NAME = "GAC_project_db"

In [5]:
act_hetero = [
    ds
    for ds in bw.Database(DB_NAME)
    if "1-(naphthalen-1-yl)-1H-imidazole" in ds["name"] and "Cu/PCN" in ds["name"]
][0]
act_hetero


'1-(naphthalen-1-yl)-1H-imidazole production with heterogeneous Cu/PCN catalyst (10.8 wt% Cu)' (kilogram, GLO, None)

In [6]:
act_homogen = [
    ds
    for ds in bw.Database(DB_NAME)
    if "1-(naphthalen-1-yl)-1H-imidazole" in ds["name"]
    and "Cu2O/Altman-Buchwald-ligand" in ds["name"]
][0]
act_homogen


'1-(naphthalen-1-yl)-1H-imidazole production with homogeneous Cu2O/Altman-Buchwald-ligand catalytic complex' (kilogram, GLO, None)

In [7]:
# bw.Database(DB_NAME).random()
#TODO should be accepting `methods=str`. Correct

# prbs.pretty_lca(
#     FUs=[{act: 1}],
#     methods=ipcc_methods,
# )

### Functions

In [8]:
def print_recursive_calculation(activity, lcia_method, lca_obj=None, total_score=None, amount=1, level=0, max_level=3, cutoff=1e-2):
    """"Taken from https://github.com/brightway-lca/brightway2/blob/master/notebooks/Contribution%20analysis%20and%20comparison.ipynb"""
    if lca_obj is None:
        lca_obj = bc.LCA({activity: amount}, lcia_method)
        lca_obj.lci()
        lca_obj.lcia()
        total_score = lca_obj.score
    elif total_score is None:
        raise ValueError
    else:
        lca_obj.redo_lcia({activity: amount})
        if abs(lca_obj.score) <= abs(total_score * cutoff):
            return
    print("{}{:4.3f} ({:06.4f}): {:.70}".format("  " * level, lca_obj.score / total_score, lca_obj.score, str(activity)))
    if level < max_level:
        for exc in activity.technosphere():
            print_recursive_calculation(
                activity=exc.input, 
                lcia_method=lcia_method, 
                lca_obj=lca_obj, 
                total_score=total_score, 
                amount=amount * exc['amount'], 
                level=level + 1, 
                max_level=max_level, 
                cutoff=cutoff
            )    

In [9]:
def get_named_edges(traversed_dict: Dict, act: bd.backends.peewee.Activity) -> List:
    rev_activity_dict, _, _ = traversed_dict["lca"].reverse_dict()
    named_edges = []
    for edge in traversed_dict["edges"]:
        named_edge = {"impact": edge["impact"]}

        act_from = bw.get_activity(rev_activity_dict[edge["from"]])

        named_edge["from"] = act_from["name"] # + " (" + act_from["location"] + ")"
        if edge["to"] == -1:
            named_edge["to"] = act["name"] + "(start)"
        else:
            act_to = bw.get_activity(rev_activity_dict[edge["to"]])
            named_edge["to"] = act_to["name"] # + " (" + act_to["location"]  + ")"
        named_edges.append(named_edge)
    return named_edges


In [10]:
def prepare_dataframe(lst: List[dict], group: bool = True) -> pd.DataFrame:
    to_return = pd.DataFrame(lst)
    to_return = to_return[["to", "from", "impact"]].rename(
        columns={"to": "target", "from": "source", "impact": "value"}
    )
    if group:
        to_return = to_return.groupby(["target", "source"])["value"].sum().reset_index()
    return to_return


In [11]:
def plot_sankey(
    df_links: pd.DataFrame, size: Tuple, save: bool = True, file_name: str = "test"
):
    links = df_links.to_dict("records")
    layout = Layout(width=size[0], height=size[1])
    sankey = SankeyWidget(
        links=links,
        linkLabelFormat=".1f",
        layout=layout,
        align_link_types=True,
        # margins=dict(top=10, bottom=0, left=250, right=300),
    )
    if save:
        sankey.auto_save_svg(f"../data/interim/svg_files/{file_name}.svg")
    return sankey


## Print before Graph Traversal

In [12]:
print_recursive_calculation(activity=act_hetero, lcia_method=ipcc_methods)

1.000 (5.6098): '1-(naphthalen-1-yl)-1H-imidazole production with heterogeneous Cu/PCN
  0.386 (2.1631): '1-iodonaphthalene production' (kilogram, GLO, None)
    0.386 (2.1631): 'market for naphthalene sulfonic acid' (kilogram, GLO, None)
      0.110 (0.6177): 'naphthalene sulfonic acid production' (kilogram, RER, None)
      0.260 (1.4611): 'naphthalene sulfonic acid production' (kilogram, RoW, None)
  0.487 (2.7300): 'market for imidazole' (kilogram, GLO, None)
    0.352 (1.9760): 'imidazole production' (kilogram, RoW, None)
      0.034 (0.1892): 'market for ammonia, anhydrous, liquid' (kilogram, CN, None)
      0.060 (0.3385): 'market for ammonia, anhydrous, liquid' (kilogram, RoW, None)
      0.012 (0.0663): 'market group for electricity, medium voltage' (kilowatt hour, RAS, No
      0.031 (0.1711): 'market for formaldehyde' (kilogram, RoW, None)
      0.171 (0.9603): 'market for glyoxal' (kilogram, RoW, None)
      0.015 (0.0840): 'market for heat, from steam, in chemical industry

In [13]:
print_recursive_calculation(activity=act_homogen, lcia_method=ipcc_methods)

1.000 (70.3327): '1-(naphthalen-1-yl)-1H-imidazole production with homogeneous Cu2O/Alt
  0.028 (2.0021): '1-iodonaphthalene production' (kilogram, GLO, None)
    0.028 (2.0021): 'market for naphthalene sulfonic acid' (kilogram, GLO, None)
      0.019 (1.3523): 'naphthalene sulfonic acid production' (kilogram, RoW, None)
  0.036 (2.5268): 'market for imidazole' (kilogram, GLO, None)
    0.026 (1.8289): 'imidazole production' (kilogram, RoW, None)
      0.013 (0.8888): 'market for glyoxal' (kilogram, RoW, None)
  0.771 (54.2100): 'Cu2O/Altman-Buchwald-ligand catalytic complex production' (kilogram, 
    0.767 (53.9492): '4,7-Dimethoxy-1,10-phenanthroline (Altman-Buchwald ligand) production
      0.011 (0.8021): 'market for acetic acid, without water, in 98% solution state' (kilogr
      0.012 (0.8247): 'market for acetic acid, without water, in 98% solution state' (kilogr
      0.544 (38.2841): 'market for trichloromethane' (kilogram, RER, None)
      0.016 (1.1572): 'market for aniline

## Graph Traversal

### Documentation regarding Graph Traversal

```python
t = bw.GraphTraversal().calculate(demand={act:1}, method=ipcc_methods[1])
t.keys()
>>> dict_keys(['nodes', 'edges', 'lca', 'counter'])
# ---
t["nodes"][-1]
>>>  {'amount': 1, 'cum': 9.327382788679612, 'ind': 9.327382788679611e-06}`
```

from [bw documentation](https://2.docs.brightway.dev/lca.html#graph-traversal):  
`t["nodes"]` - dictionary (each node = row index in `A` matrix)
- `amount`: The total amount of this node needed to produce the functional unit
- `cum`: The cumulative LCA impact score attributable to the needed amount of this node, including its specific supply chain.
- `ind`: The individual LCA impact score directly attributable to one unit of this node, i.e. the score from the direct emissions and resource consumption of this node.

```python
t["edges"][0]
>>> {'to': -1,
 'from': 21263,
 'amount': 1,
 'exc_amount': 1,
 'impact': 9.327382788679609}
```

from [bw documentation](https://2.docs.brightway.dev/lca.html#graph-traversal):  
``t["edges"]`` - list  
- ``to``: The row index of the node consuming the product.
- ``from``: The row index of the node producing the product.
- ``amount``: The total amount of product from needed for the amount of to needed.
- ``exc_amount``: The amount of from needed for one unit of to, i.e. the value given in the technosphere matrix.
- ``impact``: The total LCA impact score embodied in this edge, i.e. the individual score of from times amount.

In [14]:
t_hetero = bw.GraphTraversal().calculate(demand={act_hetero: 1}, method=ipcc_methods, cutoff=0.05)
print(f"Calculated {t_hetero['counter']} LCAs.")

# Named edges in a list
ne_hetero = get_named_edges(traversed_dict=t_hetero, act=act_hetero)
# Clean dataframe with renamed columns
df_hetero = prepare_dataframe(lst=ne_hetero, group=True)


Calculated 404 LCAs.


In [15]:
t_homogen = bw.GraphTraversal().calculate(demand={act_homogen: 1}, method=ipcc_methods, cutoff=0.028)
print(f"Calculated {t_homogen['counter']} LCAs.")

# Named edges in a list
ne_homogen = get_named_edges(traversed_dict=t_homogen, act=act_homogen)
# Clean dataframe with renamed columns
df_homogen = prepare_dataframe(lst=ne_homogen, group=True)


Calculated 716 LCAs.


In [16]:
# un = ba.GTManipulator().unroll_graph(
#     nodes=t_hetero["nodes"], edges=t_hetero["edges"], score=t_hetero["nodes"][-1]["cum"]
# )

# un[0].keys()
# un[2]

## Plot results

In [17]:
# Layout size in mm
mm_to_px = 3.7795275591

w_h_in_mm = (470,120)
# w_h_in_mm = (200,70)
(w_px, h_px) = str(w_h_in_mm[0]*mm_to_px), str(w_h_in_mm[1]*mm_to_px)
(w_px, h_px)

('1776.377952777', '453.54330709199996')

In [18]:
plot_sankey(
    df_links=df_hetero[df_hetero.target != df_hetero.source],
    size=(w_px, h_px),
    save=True,
    file_name="hetero",
)


SankeyWidget(align_link_types=True, layout=Layout(height='453.54330709199996', width='1776.377952777'), linkLa…

In [19]:
df_hetero

,target,source,value
0,1-(naphthalen-1-yl)-1H-imidazole production wi...,1-iodonaphthalene production,2.163055
1,1-(naphthalen-1-yl)-1H-imidazole production wi...,"Cu/PCN catalyst production, at 10.8 wt% Cu",0.402700
2,1-(naphthalen-1-yl)-1H-imidazole production wi...,market for dimethyl sulfoxide,0.314026
3,1-(naphthalen-1-yl)-1H-imidazole production wi...,market for imidazole,2.729986
4,1-(naphthalen-1-yl)-1H-imidazole production wi...,1-(naphthalen-1-yl)-1H-imidazole production wi...,5.609766
5,1-iodonaphthalene production,market for naphthalene sulfonic acid,2.163055
6,"Cu/PCN catalyst production, at 10.8 wt% Cu",polymeric carbon nitride prodution,0.330835
7,"air separation, cryogenic","market group for electricity, medium voltage",0.251784
8,"calcium carbide production, technical grade","market group for electricity, medium voltage",0.074602
9,coking,market for hard coal,0.318650


In [20]:
plot_sankey(
    df_links=df_homogen[df_homogen.target != df_homogen.source],
    size=(w_px, h_px),
    save=True,
    file_name="homogen",
)


SankeyWidget(align_link_types=True, layout=Layout(height='453.54330709199996', width='1776.377952777'), linkLa…

In [21]:
df_homogen[
    df_homogen.source.isin(["market group for electricity, medium voltage"])
].sort_values("value", ascending=False)


,target,source,value
53,"market group for electricity, medium voltage","market group for electricity, medium voltage",14.787129
18,"air separation, cryogenic","market group for electricity, medium voltage",2.297423
28,"chlor-alkali electrolysis, membrane cell","market group for electricity, medium voltage",2.133812
59,trichloromethane production,"market group for electricity, medium voltage",1.562060
14,N-methyl-2-pyrrolidone production,"market group for electricity, medium voltage",0.357757
26,"butane-1,4-diol production","market group for electricity, medium voltage",0.333475
33,cumene production,"market group for electricity, medium voltage",0.300437
36,"dehydrogenation of butan-1,4-diol","market group for electricity, medium voltage",0.270366
56,"phenol production, from cumene","market group for electricity, medium voltage",0.243044
30,"chlorine production, liquid","market group for electricity, medium voltage",0.237712


In [22]:
df_homogen[
    df_homogen.source.isin(["market group for natural gas, high pressure"])
].sort_values("value", ascending=False)


,target,source,value
61,trichloromethane production,"market group for natural gas, high pressure",1.163064
38,"heat production, natural gas, at industrial fu...","market group for natural gas, high pressure",1.131604


In [23]:
df_homogen[
    df_homogen.target.isin(["trichloromethane production"])
].sort_values("value", ascending=False)

,target,source,value
60,trichloromethane production,"market group for heat, district or industrial,...",5.784491
58,trichloromethane production,"market for chlorine, liquid",5.032062
59,trichloromethane production,"market group for electricity, medium voltage",1.562060
61,trichloromethane production,"market group for natural gas, high pressure",1.163064
